In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from pathlib import Path

import re

# What's happening here?

In this notebook we will read in all the csv files and compile it to useful information. 
We will end up with comprehensive CSV files containing all usefull information. 

Add to this list:
- Responses
- RT
- Testmoment
- Ppt number
- Total time taken


Make list
Check for all instances with that name
if exists: wCam
- get that
else:
- get other



# Get Data

In [2]:
# Declare all directories:

data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))

# TESTMOMENT A

In [21]:
os.chdir(data_A_dir)
ACsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
ACsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(ACsv):
    ACsvID.append(ACsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in ACsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
ACsvID = set(ACsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [22]:
exclude = ['203', # Double moment A
           '262', # Double moment A
           '285', # Double moment A (both wCAM)
           '334', # Double moment A
           '312', # Double moment A
           '350', # Gives some weird stuff later on
           '361', # Double moment A, but second time moment A also did moment B (2 days later).
           '531', # Double moment A
           '661', # Double moment A, but second time moment A also did moment B.
           '667', # Double moment A, but second time moment A also did moment B. Only 4 days between moment A and B, figure out # i might have been wrong based on video. Almost month later
           
           '105', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '106', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '107', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '111', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '118', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '119', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '123', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '124', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '125', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '127', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '999', # No demographics present. Last known participant number is 677. So probably a testing round
           '998', # No demographics present. Last known participant number is 677. So probably a testing round
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           
           '373', # Has no proper experiment order - check in R file
           
           '66_', # Weird participant number, let's see if we can figure out what the correct one would've been here -> change name of csv/webm
           '6_1', # Very hacky, because if the particpiant ends in '6' and date starts with '1' this will also get triggered. Figure out how this ends up in the csv file and try to solve
           '&à&', # Weird stuff, shouldn't be here
           '11_', # Probably the same as the others, check it out
          ]

In [23]:
index = 0
for pptnum in tqdm(ACsvID): # Loop through all file ID's
#     print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(ACsv[indices[0]])
            date = re.search(pptnum + '_(.*).csv', ACsv[indices[0]]).group(1)
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(ACsv[indices[0]])
                date = re.search(pptnum + '_(.*)_wCAM.csv', ACsv[indices[0]]).group(1)
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe
            # Get deduced indices for startviewing and endviewing
            indicesSTARTView= [x - 2 for x in indices2]
            indicesSTOPView= [x - 1 for x in indices2]
            
            # Make start and endbeep indices
            beepindex = [i for i, s in enumerate(data['trial_type']) if "audio-keyboard-response" in s]
            startBeep = beepindex[0]
            endBeep = beepindex[1]

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment A, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment A, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")
            
            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
                                       [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, [demographics['Isala'][demoindex].values[0]] * stimNum,
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['A'] * stimNum, [date] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], 
                                       data['time_elapsed'][indicesSTARTView], data['time_elapsed'][indicesSTOPView], data['time_elapsed'][indices2], [data['time_elapsed'][startBeep]] * stimNum, [data['time_elapsed'][endBeep]] * stimNum, )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
                                     'Exclusie', 'TrueFollicular', 'TrueLuteal', 'Isala',
                                     'Order', 'Moment', 'Date', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt',
                                     'startView', 'stopView', 'stopResponse', 'startBeep', 'endBeep'])
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
#             break
# dataset
datasetA = dataset

 87%|█████████████████████████████████████████████████████████████████████▌          | 342/393 [00:24<00:03, 14.17it/s]

No demographics entries for ppt 126.


# TESTMOMENT B

In [17]:
os.chdir(data_B_dir)
BCsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BCsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(BCsv):
    BCsvID.append(BCsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in BCsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
BCsvID = set(BCsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [18]:
exclude = ['100', # No demographics present. First known participant number is 129.
           '106', # No demographics present. First known participant number is 129. 
           '119', # No demographics present. First known participant number is 129. 
           '125', # No demographics present. First known participant number is 129.
           '126', # No demographics present. First known participant number is 129.
           '127', # No demographics present. First known participant number is 129.
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '789', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           
           
           '350', # Double moment B, but also 1 moment A. Two moment B's 5 days apart. Weird. Figure out
           '361', # Messed it up and made 4_.ipynb crash
           '141', # Double moment B - both wCAM
           '405', # Double moment B
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           '488', # Is 4888, so not yet able to include. Retrace where the typo comes from -> check via date probably
           '459', # Same as above; is 4590
           
           '590', # Kicked out for now, because '4590' is also present (which is wrong). When 4590 is resolved, 590 can come back
           '667', # Kicked out because also '6677' present. So someone did something wrong here. Probably need to kik out 6677 (rename?) -> figure out
           
           
           '14_', # Weird stuff
           '_28', # Weird stuff
           '66_', # Weird stuff
           
           '987', # Trial run probably
           '995', # No demographics, Trial run probably
           '996', # Trial run probably
           '997', # Trial run probably
           '999', # Trial run probably
          ]

In [20]:
index = 0
for pptnum in tqdm(BCsvID): # Loop through all file ID's
#     print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(BCsv[indices[0]])
            date = re.search(pptnum + '_(.*).csv', BCsv[indices[0]]).group(1)
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(BCsv[indices[0]])
                date = re.search(pptnum + '_(.*)_wCAM.csv', BCsv[indices[0]]).group(1)
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe
            # Get deduced indices for startviewing and endviewing
            indicesSTARTView= [x - 2 for x in indices2]
            indicesSTOPView= [x - 1 for x in indices2]
            
            # Make start and endbeep indices
            beepindex = [i for i, s in enumerate(data['trial_type']) if "audio-keyboard-response" in s]
            startBeep = beepindex[0]
            endBeep = beepindex[1]

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment B, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment B, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")

            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
                                       [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, [demographics['Isala'][demoindex].values[0]] * stimNum,
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['B'] * stimNum, [date] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], 
                                       data['time_elapsed'][indicesSTARTView], data['time_elapsed'][indicesSTOPView], data['time_elapsed'][indices2], [data['time_elapsed'][startBeep]] * stimNum, [data['time_elapsed'][endBeep]] * stimNum )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
                                     'Exclusie', 'TrueFollicular', 'TrueLuteal', 'Isala',
                                     'Order', 'Moment', 'Date', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt',
                                     'startView', 'stopView', 'stopResponse', 'startBeep', 'endBeep'])
            
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
    #         break
# dataset
datasetB = dataset

100%|████████████████████████████████████████████████████████████████████████████████| 381/381 [00:28<00:00, 13.17it/s]


In [10]:
dataset = pd.concat([datasetA, datasetB], ignore_index=True)
dataset

,ID,Age,FirstMenstrual,MenstrualDuration,Exclusie,TrueFollicular,TrueLuteal,Isala,Order,Moment,...,BSRI,Stimulus,Valence,Arousal,rt,startView,stopView,stopResponse,startBeep,endBeep
0,363,24,13,31,NaN,6-1-2021,23-1-2021,NaN,B-A,A,...,333.0,5600.0,87.0,69.0,13108.325,248089,254093,267212,242314,610403
1,363,24,13,31,NaN,6-1-2021,23-1-2021,NaN,B-A,A,...,333.0,9600.0,14.0,65.0,6391.565,270221,276223,282617,242314,610403
2,363,24,13,31,NaN,6-1-2021,23-1-2021,NaN,B-A,A,...,333.0,9592.0,27.0,71.0,6235.585,285625,291626,297864,242314,610403
3,363,24,13,31,NaN,6-1-2021,23-1-2021,NaN,B-A,A,...,333.0,4180.0,33.0,50.0,10417.800,300872,306874,317295,242314,610403
4,363,24,13,31,NaN,6-1-2021,23-1-2021,NaN,B-A,A,...,333.0,7480.0,67.0,31.0,5641.140,320301,326303,331947,242314,610403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14801,346,23,12,26,NaN,28-12-2020,9-1-2021,NaN,A-B,B,...,415.0,1750.0,67.0,19.0,3662.435,335479,341487,345154,119391,394246
14802,346,23,12,26,NaN,28-12-2020,9-1-2021,NaN,A-B,B,...,415.0,9584.0,18.0,72.0,2952.810,348162,354166,357124,119391,394246
14803,346,23,12,26,NaN,28-12-2020,9-1-2021,NaN,A-B,B,...,415.0,5800.0,97.0,20.0,3564.495,360138,366147,369716,119391,394246
14804,346,23,12,26,NaN,28-12-2020,9-1-2021,NaN,A-B,B,...,415.0,2070.0,62.0,80.0,3211.265,372722,378723,381939,119391,394246


In [11]:
len(dataset)/44

336.5

In [12]:
os.chdir(os.path.join(data_dir,date_dir))
dataset.to_csv('allPMSdata.csv')

In [13]:
dataset.loc[:,'avg_PD_left'] = 0

In [15]:
dataset.ID == 192

0        False
1        False
2        False
3        False
4        False
         ...  
14801    False
14802    False
14803    False
14804    False
14805    False
Name: ID, Length: 14806, dtype: bool